# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:

# import libraries
import os
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\janar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
database_filepath = "../data/disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name,engine)


In [12]:
X = df['message']
y = df.iloc[:, 4:]

In [13]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [14]:
import nltk
import re
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\janar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:

X_train, X_test, y_train, y_test = train_test_split(X,y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002B4BF5B40D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
y_pred = pipeline.predict(X_test)

In [19]:
def test_model(y_test, y_pred):
    
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [20]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.76      0.27      0.40      1566
           1       0.80      0.97      0.88      4949
           2       1.00      0.10      0.19        39

    accuracy                           0.80      6554
   macro avg       0.86      0.45      0.49      6554
weighted avg       0.80      0.80      0.76      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5432
           1       0.86      0.42      0.56      1122

    accuracy                           0.89      6554
   macro avg       0.87      0.70      0.75      6554
weighted avg       0.88      0.89      0.87      6554

offer               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

water               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6136
           1       0.90      0.29      0.44       418

    accuracy                           0.95      6554
   macro avg       0.93      0.64      0.71      6554
weighted avg       0.95      0.95      0.94      6554

food               precision    recall  f1-score   support

           0       0.94      0.99      0.96      5858
           1       0.86      0.42      0.57       696

    accuracy                           0.93      6554
   macro avg       0.90      0.71      0.77      6554
weighted avg       0.93      0.93      0.92      6554

shelter               precision    recall  f1-score   support

           0       0.94      1.00      0.97      5998
           1       0.84      0.28      0.43       556

    accuracy                           0.93      6554
   macro avg       0.89      0.64      0.70      6554
weighted avg       0.93      0.93      0.92      6554



C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

tools               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.00      0.00      0.00        36

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

hospitals               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       0.00      0.00      0.00        70

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

shops               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000002B4BF5B40D0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000002B4BF5B40D0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [26]:
param_grid = {
    'clf__estimator__n_estimators' : [50, 100]
}
cv = GridSearchCV(pipeline, param_grid, verbose=3)
cv


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002B4BF5B40D0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]}, verbose=3)

In [27]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ...clf__estimator__n_estimators=50;, score=0.234 total time= 3.5min
[CV 2/5] END ...clf__estimator__n_estimators=50;, score=0.233 total time= 3.9min
[CV 3/5] END ...clf__estimator__n_estimators=50;, score=0.236 total time= 3.3min
[CV 4/5] END ...clf__estimator__n_estimators=50;, score=0.239 total time= 3.5min
[CV 5/5] END ...clf__estimator__n_estimators=50;, score=0.235 total time= 3.5min
[CV 1/5] END ..clf__estimator__n_estimators=100;, score=0.235 total time= 6.6min
[CV 2/5] END ..clf__estimator__n_estimators=100;, score=0.238 total time= 5.4min
[CV 3/5] END ..clf__estimator__n_estimators=100;, score=0.235 total time= 6.4min
[CV 4/5] END ..clf__estimator__n_estimators=100;, score=0.241 total time= 7.1min
[CV 5/5] END ..clf__estimator__n_estimators=100;, score=0.236 total time= 5.0min


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002B4BF5B40D0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]}, verbose=3)

In [28]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
y_pred = cv.predict(X_test)
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.74      0.28      0.41      1566
           1       0.81      0.97      0.88      4949
           2       1.00      0.10      0.19        39

    accuracy                           0.80      6554
   macro avg       0.85      0.45      0.49      6554
weighted avg       0.79      0.80      0.76      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5432
           1       0.86      0.42      0.56      1122

    accuracy                           0.89      6554
   macro avg       0.87      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554

offer               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

water               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6136
           1       0.91      0.25      0.39       418

    accuracy                           0.95      6554
   macro avg       0.93      0.62      0.68      6554
weighted avg       0.95      0.95      0.94      6554

food               precision    recall  f1-score   support

           0       0.94      0.99      0.96      5858
           1       0.85      0.43      0.57       696

    accuracy                           0.93      6554
   macro avg       0.89      0.71      0.77      6554
weighted avg       0.93      0.93      0.92      6554

shelter               precision    recall  f1-score   support

           0       0.94      1.00      0.96      5998
           1       0.85      0.26      0.39       556

    accuracy                           0.93      6554
   macro avg       0.89      0.63      0.68      6554
weighted avg       0.93      0.93      0.92      6554



C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

buildings               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6220
           1       0.89      0.05      0.10       334

    accuracy                           0.95      6554
   macro avg       0.92      0.53      0.54      6554
weighted avg       0.95      0.95      0.93      6554

electricity               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6400
           1       1.00      0.01      0.03       154

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.97      6554

tools               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.00      0.00      0.00        36

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99   

C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\janar\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

fire               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6494
           1       0.00      0.00      0.00        60

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

earthquake               precision    recall  f1-score   support

           0       0.97      0.99      0.98      5970
           1       0.91      0.73      0.81       584

    accuracy                           0.97      6554
   macro avg       0.94      0.86      0.90      6554
weighted avg       0.97      0.97      0.97      6554

cold               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6415
           1       0.88      0.05      0.10       139

    accuracy                           0.98      6554
   macro avg       0.93      0.53      0.54      6554
weighted avg       0.98      0.98      0.97      6554

In [30]:

accuracy = (y_pred == y_test).mean()
accuracy

related                   0.800427
request                   0.888618
offer                     0.994965
aid_related               0.769759
medical_help              0.925389
medical_products          0.954532
search_and_rescue         0.974367
security                  0.981691
military                  0.966738
child_alone               1.000000
water                     0.950412
food                      0.931492
shelter                   0.933018
clothing                  0.986573
money                     0.978486
missing_people            0.989930
refugees                  0.970095
death                     0.963686
other_aid                 0.875191
infrastructure_related    0.939884
transport                 0.956363
buildings                 0.951327
electricity               0.976808
tools                     0.994507
hospitals                 0.989319
shops                     0.996033
aid_centers               0.988709
other_infrastructure      0.959567
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [31]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.